In [1]:
#IMPORTO LIBRERIE
import pandas as pd

In [2]:
labeled_df = pd.read_csv('labeled_df.csv')
labeled_df.shape

(90960, 16)

In [3]:
labeled_df = labeled_df.drop(columns='Unnamed: 0')
labeled_df.head()

In [5]:
from sklearn.ensemble import IsolationForest
X=labeled_df.copy()
X=X.drop(columns=['businessName','customerId','timestamp'])
model=IsolationForest(random_state=0,contamination=0.001)
model.fit(X)
print(model.get_params())

C:\Users\Robivad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


{'bootstrap': False, 'contamination': 0.001, 'max_features': 1.0, 'max_samples': 'auto', 'n_estimators': 100, 'n_jobs': None, 'random_state': 0, 'verbose': 0, 'warm_start': False}


In [6]:
import pickle
pickle.dump(model, open('IF_model.pkl','wb'))

In [7]:
encoder_dict=pickle.load(open('enc_dict.pkl','rb'))
columns_to_label = ['country', 'userAgent_browserFamily', 'userAgent_osFamily', 'IBAN_countryCode', 'IBAN_bankCode','CAP', 'SIA', 'SAE', 'RAE','companyType']

In [8]:
def inverse_and_scores(if_df,labeled_df,enc_dict,columns,if_model):
    df_copy=labeled_df.copy()
    scores=if_model.decision_function(if_df)
    anomaly_score=if_model.predict(if_df)

    for clmn in columns:
        df_copy[clmn]=enc_dict[clmn].inverse_transform(df_copy[clmn])

    df_copy['scores'] = scores
    df_copy['anomaly_score'] = anomaly_score

    return df_copy

df_scores=inverse_and_scores(X,labeled_df,encoder_dict,columns_to_label,model)

df_scores.head()

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores,anomaly_score
0,93000801,2021-09-15 16:13:35.147,Italy,150.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0.153537,1
1,93000801,2021-08-24 10:34:24.487,Greece,3200.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0.127344,1
2,93000801,2021-09-14 16:21:43.33,Italy,8350.00,c7543b6f04850eae288aa52783b33363,1,Firefox,Windows,IT,5856,CLFUY,33077,492,630,SAS,0.136098,1
3,93001001,2021-06-30 09:07:16.733,Italy,500.00,3f241dfcc38064261fec0f303c2784e5,1,Chrome,Windows,IT,2008,CEFGV,31039,430,505,SRL,0.279952,1
4,93001001,2021-07-12 16:36:02.857,Italy,1895.68,a5d1eaffa8e325d370665f43624339a1,1,Chrome,Windows,IT,3069,CEFGV,31039,430,505,SRL,0.285561,1


In [10]:
df_scores[df_scores['anomaly_score']==-1].sort_values('scores')

,customerId,timestamp,country,amount,businessName,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,scores,anomaly_score
43968,41999520,2021-09-16 11:41:45.407,Italy,270000.00,d1126a5e3988d76235cde4c7eed8ebbd,0,Other,Linux,IT,3493,347D3,39038,430,830,GMBH,-0.080374,-1
42632,45915153,2021-08-24 11:47:38.077,Italy,299.00,73322bb678a4c1b6a94fe29411a5fafe,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,-0.054135,-1
42622,45915153,2021-09-02 15:44:16.347,Italy,687.00,73322bb678a4c1b6a94fe29411a5fafe,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,-0.052696,-1
42623,45915153,2021-04-29 14:17:14.26,Italy,1000.00,f2a85ebb8645443730a3fba554a69914,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,-0.051738,-1
42619,45915153,2021-07-30 15:25:38.557,Italy,1000.00,f2a85ebb8645443730a3fba554a69914,0,Other,Linux,IT,8904,082X3,30024,614,342,DI,-0.051738,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35718,37138459,2021-04-28 17:31:45.617,Italy,1200.00,e2390d68c5d65e82e64c907234ea9497,0,Other,Linux,IT,5856,AJ822,39100,482,419,SNC,-0.000376,-1
35699,37138459,2021-05-31 12:26:50.96,Italy,1000.00,8fa0dc37670dc010628473c668cc74c0,0,Other,Linux,IT,5856,AJ822,39100,482,419,SNC,-0.000376,-1
35709,37138459,2021-05-31 12:27:54.693,Italy,1200.00,e2390d68c5d65e82e64c907234ea9497,0,Other,Linux,IT,5856,AJ822,39100,482,419,SNC,-0.000376,-1
52883,58147956,2021-09-08 09:05:32.347,Italy,554.50,ce9ac220c8cff10ee86a7bad7708142f,1,Safari,Mac OS X,IT,8904,CRUT0,31010,615,11,SS,-0.000326,-1


In [ ]:
df_scores.to_csv('IF_df_scores.csv')

In [31]:
IF_labeled_df = X.copy()
IF_labeled_df['label']=model.predict(X)
IF_labeled_df['label']=IF_labeled_df['label'].replace(1,0)
IF_labeled_df['label']=IF_labeled_df['label'].replace(-1,1)
IF_labeled_df.head()

,country,amount,userAgent_isPc,userAgent_browserFamily,userAgent_osFamily,IBAN_countryCode,IBAN_bankCode,SIA,CAP,SAE,RAE,companyType,label
0,6,150.00,1,4,5,17,699,2013,144,19,113,5,0
1,5,3200.00,1,4,5,17,699,2013,144,19,113,5,0
2,6,8350.00,1,4,5,17,699,2013,144,19,113,5,0
3,6,500.00,1,0,5,17,123,1781,90,8,99,10,0
4,6,1895.68,1,0,5,17,285,1781,90,8,99,10,0


In [32]:
IF_labeled_df.to_csv('IF_labeled_df.csv')